In [1]:
import sys
# sys.path.insert(0, os.path.abspath('..'))
# import os
# folder = os.getcwd().split("\\notebooks")[0]
sys.path.append('..')

from transport_frames.graphbuilder.graph import Graph 
from transport_frames.frame_grader.advanced_grade import AdvancedGrader
from transport_frames.framebuilder.frame import Frame
import transport_frames.utils.helper_funcs as he

import osmnx as ox
import geopandas as gpd
import networkx as nx
from shapely import Polygon, MultiPolygon
import shapely
from iduedu import get_adj_matrix_gdf_to_gdf
from iduedu import get_intermodal_graph


/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
geocode=81993
polygon = ox.geocode_to_gdf(f'R{geocode}',by_osmid=True)
local_crs = 32637

In [3]:
tula = he.convert_geometry_from_wkt(nx.read_graphml('/Users/polina/Desktop/github/transport_frames/transport_frames/tula.graphml'))
tula = he.convert_list_attr_from_str(tula)
g =  Graph(tula,crs=local_crs)

2024-10-25 12:35:37.571 | INFO     | transport_frames.graphbuilder.graph:_prepare_attrs:133 - Preparing the graph...
2024-10-25 12:35:46.798 | INFO     | transport_frames.graphbuilder.graph:_prepare_attrs:171 - Graph is ready!


In [8]:
import momepy


n,e = momepy.nx_to_gdf(graf)

In [5]:
# polygon regions cities
regions = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/lo_gdfs/regions_of_russia.geojson')
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions[regions['geometry'].apply(lambda x: isinstance(x, (Polygon, MultiPolygon)))]

admin_centers = gpd.read_file('/Users/polina/Desktop/github/transport_frames/data/Тульская_область/Тульская_область_region_points.geojson')
admin_centers = admin_centers[admin_centers.is_admin_centre_district==1][['name','geometry']].reset_index()


In [6]:
f = Frame(graph=g.graph, regions=regions.to_crs(local_crs), polygon=polygon.to_crs(local_crs))


TypeError: Frame.__init__() missing 1 required positional argument: 'centers'

In [7]:
import pickle
with open("tulaframe.pickle", 'rb') as f:  # notice the r instead of w
    graf = pickle.load(f)

In [19]:
import networkx as nx
import pandas as pd
import geopandas as gpd
from shapely.geometry import LineString

# Assuming 'graf' is your existing NetworkX graph
# For example, let's say 'graf' is already defined and populated with edges and node attributes

# Identify connected components
components = nx.connected_components(nx.Graph(graf))
component_mapping = {}
for i, component in enumerate(components):
    for node in component:
        component_mapping[node] = i

# Create edge DataFrame with component information
edges = [{'source': u, 'target': v, 'component': component_mapping[u]} for u, v in graf.edges()]
edge_df = pd.DataFrame(edges)

# Create geometries for each edge using x and y attributes
def create_line_string(row):
    source_pos = (graf.nodes[row['source']]['x'], graf.nodes[row['source']]['y'])
    target_pos = (graf.nodes[row['target']]['x'], graf.nodes[row['target']]['y'])
    return LineString([source_pos, target_pos])

edge_df['geometry'] = edge_df.apply(create_line_string, axis=1)

# Convert to GeoDataFrame
edge_gdf = gpd.GeoDataFrame(edge_df, geometry='geometry')

# Set CRS if necessary (for example, WGS84)
# edge_gdf.set_crs(epsg=4326, inplace=True)

# print(edge_gdf)

In [34]:
segment = [k for k,v in component_mapping.items() if v==40]

In [36]:
H = graf.subgraph(segment).copy() 

In [38]:
list(g.graph.edges(data=True))[0]

(0,
 1,
 {'length_meter': 127.64,
  'geometry': <LINESTRING (443580.422 5883013.035, 443543.814 5883135.313)>,
  'highway': 'primary',
  'ref': nan,
  'reg': 2,
  'maxspeed': 22.22222222222222,
  'time_min': 0.096,
  'type': 'car'})

In [37]:
pd.concat(momepy.nx_to_gdf(H)).explore()